In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import re
import pickle

In [2]:
df=pd.read_excel("news.xlsx")

In [3]:
df

,Headline,Short,Source,Time,Publish Date
0,4 ex-bank officials booked for cheating bank o...,The CBI on Saturday booked four former officia...,The New Indian Express,09:25:00,2017-03-26
1,Supreme Court to go paperless in 6 months: CJI,Chief Justice JS Khehar has said the Supreme C...,Outlook,22:18:00,2017-03-25
2,"At least 3 killed, 30 injured in blast in Sylh...","At least three people were killed, including a...",Hindustan Times,23:39:00,2017-03-25
3,Why has Reliance been barred from trading in f...,Mukesh Ambani-led Reliance Industries (RIL) wa...,Livemint,23:08:00,2017-03-25
4,Was stopped from entering my own studio at Tim...,TV news anchor Arnab Goswami has said he was t...,YouTube,23:24:00,2017-03-25
...,...,...,...,...,...
55099,Sensex loses 400 points to hit 52-week low,"Tracking weak cues from the Asian markets, the...",The Financial Express,10:36:00,2016-01-20
55100,China to inject $91 bn into the money markets,Amid growing concerns about China&#39;s econom...,Reuters,12:06:00,2016-01-20
55101,Ghulam Ali set to make acting debut in Bollywood,Pakistani Ghazal singer Ghulam Ali will soon m...,Hindustan Times,12:10:00,2016-01-20
55102,IS acknowledges death of Jihadi John: Report,The Islamic State (IS) has acknowledged the de...,YouTube,11:53:00,2016-01-20


In [4]:
df.drop(['Source ', 'Time ', 'Publish Date'], axis=1, inplace=True)

In [5]:
news = df['Short']
summary = df['Headline']

In [6]:
news[20], summary[20]

('The Indian Army, after consultations with the Defence Ministry, is considering cutting down the number of its sahayaks by 25%, which translates to 10,000 jawans. This comes after videos of soldiers alleging that they were often made to perform menial tasks for the officers surfaced online. The sahayak jawans will be moved to army formations and replaced by civilians.',
 'Indian Army considering to reduce its &#39;sahayaks&#39; by 10,000')

In [7]:
summary = summary.apply(lambda x: '<go> ' + x + ' <stop>')
summary.head()

0    <go> 4 ex-bank officials booked for cheating b...
1    <go> Supreme Court to go paperless in 6 months...
2    <go> At least 3 killed, 30 injured in blast in...
3    <go> Why has Reliance been barred from trading...
4    <go> Was stopped from entering my own studio a...
Name: Headline, dtype: object

In [8]:
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
oov_token = '<unk>'

In [9]:
news_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token)

In [10]:
news_tokenizer.fit_on_texts(news)
summary_tokenizer.fit_on_texts(summary)

In [11]:
inputs = news_tokenizer.texts_to_sequences(news)
targets = summary_tokenizer.texts_to_sequences(summary)

In [12]:
summary_tokenizer.texts_to_sequences(["This is a test"])

[[184, 22, 12, 71]]

In [13]:
summary_tokenizer.sequences_to_texts([[184, 22, 12, 71]])

['this is a test']

In [15]:
encoder_vocab_size = len(news_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# vocab_size
encoder_vocab_size, decoder_vocab_size

(76362, 29661)